In [1]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.grid_search import GridSearchCV 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

C:\Users\NEW\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\NEW\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
X = np.genfromtxt("data/X_train.txt", delimiter=None)
Y = np.genfromtxt("data/Y_train.txt", delimiter=None)
xte = np.genfromtxt("data/X_test.txt", delimiter=None)

In [6]:
Xtr = X[:190000]
Ytr = Y[:190000]
Xva = X[190000:]
Yva = Y[190000:]

In [30]:
#Gradient boost
gb=GradientBoostingClassifier(n_estimators=500,max_depth=12,min_samples_split=12, min_samples_leaf=5)
gb.fit(Xtr,Ytr)



GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=12,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=5, min_samples_split=12,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [31]:
Ytr_score = gb.predict_proba(Xtr).T[1]
Y_score = gb.predict_proba(Xva).T[1]


print("Trainning auc",roc_auc_score(Ytr,Ytr_score))
print("Validation auc",roc_auc_score(Yva,Y_score))


Trainning auc 0.9666518820824345
Validation auc 0.8042912411930684


In [19]:
from sklearn import *
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(
    X, Y, test_size=0.2, random_state=42)

C:\Users\NEW\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [23]:
#Random forest
from sklearn.ensemble import RandomForestClassifier
random_forest_classifier = RandomForestClassifier(
n_estimators=1000, min_samples_leaf=4,max_depth= 50,oob_score=True)
classf = random_forest_classifier.fit(x_train, y_train)
Y_score = classf.predict_proba(x_validation).T[1]
Ytr_score = classf.predict_proba(x_train).T[1]


In [22]:
print("Training AUC: {}".format(roc_auc_score(y_train,Ytr_score)))
print("Validation AUC: {}".format(roc_auc_score(y_validation,Y_score)))

Training AUC: 0.9393463825871311
Validation AUC: 0.7863518619892627


In [29]:
#Ada boost
ab = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth=23,max_features=4,min_samples_split=32), n_estimators=800,learning_rate=0.001)
ab.fit(Xtr,Ytr)


#这里是 estimator 关系不大，100， 80 都差不多
# 10是 0.95 0.768
# 50 是 0.98 0.786
# Trainning auc 0.9901118657901069  120
# Validation auc 0.7886719734892589
# 150
# 0.001
#
Y_ptr = ab.predict_proba(Xtr).T[1]
Y_pr = ab.predict_proba(Xva).T[1]


print("Trainning auc",roc_auc_score(Ytr,Y_ptr))
print("Validation auc",roc_auc_score(Yva,Y_pr))

Trainning auc 0.9890538423940642
Validation auc 0.7939486074636888


In [32]:
from sklearn.ensemble import VotingClassifier

In [35]:
blend = VotingClassifier(estimators = [('rf',random_forest_classifier),
                                      ("ab",ab),
                                      ("gb",gb)],voting = "soft",
                        weights = [1,1,2])

In [36]:
blend.fit(Xtr,Ytr)


VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weig...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='soft',
         weights=[1, 1, 2])

In [37]:
Y_ptr = blend.predict_proba(Xtr).T[1]
Y_pr = blend.predict_proba(Xva).T[1]

print("Trainning auc",roc_auc_score(Ytr,Y_ptr))
print("Validation auc",roc_auc_score(Yva,Y_pr))

Trainning auc 0.9772733380984986
Validation auc 0.806222051182898


In [38]:
Y_ptr = blend.predict(Xtr)
Y_pr = blend.predict(Xva)

print("Trainning auc",roc_auc_score(Ytr,Y_ptr))
print("Validation auc",roc_auc_score(Yva,Y_pr))

C:\Users\NEW\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Trainning auc 0.8863733172036032
Validation auc 0.7014814909246463


C:\Users\NEW\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [39]:
Xte = np.genfromtxt('data/X_test.txt', delimiter=None)
Yte = np.vstack((np.arange(Xte.shape[0]), blend.predict_proba(Xte).T[1])).T

np.savetxt('Y_submit_blend.txt',Yte,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')

kaggle result AUC:0.799